# 6주차: 기초 머신러닝 - 분류

## 학습 목표
- 머신러닝의 기본 개념과 지도학습을 이해하자
- Train/Test 분할과 모델 평가 방법을 배우자
- K-Nearest Neighbors와 Decision Tree로 음원 장르 분류를 해보자
- 정확도, 혼동행렬로 모델 성능을 평가해보자

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 스타일 설정
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. 데이터 준비

### 1.1 음원 데이터셋 생성

머신러닝 실습을 위한 음원 데이터를 만들어보자.
장르별로 다른 특징을 가지도록 설계하여 분류 모델이 학습할 수 있도록 했다.

In [ ]:
# 시드 설정으로 재현 가능한 결과
np.random.seed(42)
n_tracks = 1200

# 장르 정의 (분류하기 쉽게 4개로 축소)
genres = ['Pop', 'Rock', 'Electronic', 'Classical']
genre_weights = [0.3, 0.25, 0.25, 0.2]

# 기본 데이터 구조
music_data = pd.DataFrame({
    'track_id': [f'T{i:04d}' for i in range(n_tracks)],
    'genre': np.random.choice(genres, n_tracks, p=genre_weights)
})

# 음향 특징들 생성 (장르별로 뚜렷한 특성 부여)
for genre in genres:
    mask = music_data['genre'] == genre
    count = mask.sum()
    
    if genre == 'Pop':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(120, 10, count), 80, 160)
        music_data.loc[mask, 'energy'] = np.clip(np.random.normal(0.7, 0.15, count), 0, 1)
        music_data.loc[mask, 'danceability'] = np.clip(np.random.normal(0.75, 0.1, count), 0, 1)
        music_data.loc[mask, 'valence'] = np.clip(np.random.normal(0.6, 0.2, count), 0, 1)
        
    elif genre == 'Rock':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(130, 15, count), 80, 160)
        music_data.loc[mask, 'energy'] = np.clip(np.random.normal(0.8, 0.1, count), 0, 1)
        music_data.loc[mask, 'danceability'] = np.clip(np.random.normal(0.5, 0.15, count), 0, 1)
        music_data.loc[mask, 'valence'] = np.clip(np.random.normal(0.5, 0.2, count), 0, 1)
        
    elif genre == 'Electronic':
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(128, 12, count), 80, 160)
        music_data.loc[mask, 'energy'] = np.clip(np.random.normal(0.85, 0.1, count), 0, 1)
        music_data.loc[mask, 'danceability'] = np.clip(np.random.normal(0.8, 0.1, count), 0, 1)
        music_data.loc[mask, 'valence'] = np.clip(np.random.normal(0.65, 0.15, count), 0, 1)
        
    else:  # Classical
        music_data.loc[mask, 'tempo'] = np.clip(np.random.normal(90, 20, count), 80, 160)
        music_data.loc[mask, 'energy'] = np.clip(np.random.normal(0.3, 0.15, count), 0, 1)
        music_data.loc[mask, 'danceability'] = np.clip(np.random.normal(0.2, 0.1, count), 0, 1)
        music_data.loc[mask, 'valence'] = np.clip(np.random.normal(0.4, 0.15, count), 0, 1)

print(f"음원 데이터셋 생성 완료!")
print(f"총 {n_tracks}개 트랙, {len(genres)}개 장르")
print(f"\n데이터 샘플:")
print(music_data.head())

print(f"\n장르별 분포:")
print(music_data['genre'].value_counts())

### 1.2 데이터 탐색

머신러닝을 하기 전에 항상 데이터를 먼저 살펴보자.
장르별로 특징이 어떻게 다른지 시각화해보자.

In [ ]:
# 장르별 특징 분포 시각화
features = ['tempo', 'energy', 'danceability', 'valence']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

colors = {'Pop': 'red', 'Rock': 'blue', 'Electronic': 'green', 'Classical': 'purple'}

for idx, feature in enumerate(features):
    for genre in genres:
        data = music_data[music_data['genre'] == genre][feature]
        axes[idx].hist(data, alpha=0.6, label=genre, color=colors[genre], bins=20, density=True)
    
    axes[idx].set_title(f'{feature.title()} Distribution by Genre')
    axes[idx].set_xlabel(feature.title())
    axes[idx].set_ylabel('Density')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 장르별 평균값 확인
print("장르별 특징 평균값:")
print(music_data.groupby('genre')[features].mean().round(3))

## 2. 머신러닝 기초 개념

### 2.1 지도학습이란?

**지도학습(Supervised Learning)**은 정답(레이블)이 있는 데이터로 모델을 학습시키는 방법이다.

**분류(Classification)의 목표:**
- 입력: 음원의 특징들 (tempo, energy, danceability, valence)
- 출력: 장르 예측 (Pop, Rock, Electronic, Classical)

**학습 과정:**
1. **훈련 데이터**로 모델 학습
2. **테스트 데이터**로 모델 평가
3. 성능이 좋으면 새로운 데이터 예측에 사용

### 2.2 Train/Test 분할

**왜 데이터를 나눌까?**
- **훈련 데이터**: 모델이 학습하는 데이터
- **테스트 데이터**: 모델 성능을 평가하는 데이터 (모델이 본 적 없는 데이터)

**일반적인 분할 비율:**
- 훈련: 70-80%
- 테스트: 20-30%

In [ ]:
# 특징과 타겟 분리
X = music_data[features]  # 특징들
y = music_data['genre']   # 타겟 (정답)

print("데이터 형태:")
print(f"특징 행렬 X: {X.shape}")
print(f"타겟 벡터 y: {y.shape}")

# 데이터 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n데이터 분할 완료:")
print(f"훈련 데이터: {X_train.shape[0]}개 ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"테스트 데이터: {X_test.shape[0]}개 ({X_test.shape[0]/len(X)*100:.1f}%)")

# 분할 후 장르별 분포 확인
print(f"\n훈련 데이터 장르 분포:")
print(y_train.value_counts())
print(f"\n테스트 데이터 장르 분포:")
print(y_test.value_counts())

## 3. K-Nearest Neighbors (KNN)

### 3.1 KNN의 직관적 이해

**KNN의 아이디어:**
"비슷한 특징을 가진 음원들은 같은 장르일 가능성이 높다"

**작동 원리:**
1. 새로운 음원이 들어오면
2. 가장 가까운 K개의 이웃을 찾는다
3. 이웃들의 장르 중 가장 많은 것으로 예측한다

**K값의 영향:**
- **K=1**: 가장 가까운 1개만 봄 (과적합 위험)
- **K=큰값**: 많은 이웃을 봄 (과소적합 위험)
- **보통 홀수값 사용** (동점 방지)

In [ ]:
# KNN 모델 생성 및 학습
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# 예측 수행
y_pred_knn = knn.predict(X_test)

# 정확도 계산
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN 정확도: {accuracy_knn:.3f} ({accuracy_knn*100:.1f}%)")

# 예측 예시
print(f"\n예측 예시 (처음 10개):")
comparison_df = pd.DataFrame({
    '실제': y_test.iloc[:10].values,
    'KNN예측': y_pred_knn[:10],
    '일치': y_test.iloc[:10].values == y_pred_knn[:10]
})
print(comparison_df)

### 3.2 최적의 K값 찾기

다양한 K값을 시도해보고 가장 좋은 성능을 내는 K를 찾아보자.

In [ ]:
# 다양한 K값에 대한 성능 평가
k_values = range(1, 21)
accuracies = []

for k in k_values:
    knn_temp = KNeighborsClassifier(n_neighbors=k)
    knn_temp.fit(X_train, y_train)
    y_pred_temp = knn_temp.predict(X_test)
    acc = accuracy_score(y_test, y_pred_temp)
    accuracies.append(acc)

# 최적의 K값 찾기
best_k = k_values[np.argmax(accuracies)]
best_accuracy = max(accuracies)

print(f"최적의 K값: {best_k}")
print(f"최고 정확도: {best_accuracy:.3f} ({best_accuracy*100:.1f}%)")

# 결과 시각화
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(k_values, accuracies, 'bo-', linewidth=2, markersize=8)
plt.axvline(best_k, color='red', linestyle='--', alpha=0.7, 
           label=f'Best K={best_k}')
plt.axhline(best_accuracy, color='red', linestyle='--', alpha=0.7)
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.title('KNN Performance by K Value')
plt.legend()
plt.grid(True, alpha=0.3)

# 정확도 분포
plt.subplot(1, 2, 2)
plt.hist(accuracies, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(best_accuracy, color='red', linestyle='--', linewidth=2,
           label=f'Best: {best_accuracy:.3f}')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Accuracy Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 성능 상위 5개 K값
top_k_indices = np.argsort(accuracies)[-5:][::-1]
print(f"\n상위 5개 K값:")
for i, idx in enumerate(top_k_indices):
    k_val = k_values[idx]
    acc = accuracies[idx]
    print(f"{i+1}. K={k_val}: {acc:.3f} ({acc*100:.1f}%)")

## 4. Decision Tree (의사결정나무)

### 4.1 Decision Tree의 직관적 이해

**Decision Tree의 아이디어:**
"일련의 질문을 통해 장르를 결정하자"

**예시 질문들:**
- Energy > 0.6인가? → Yes: Rock/Electronic 가능성 ↑
- Danceability > 0.7인가? → Yes: Pop/Electronic 가능성 ↑
- Tempo < 100인가? → Yes: Classical 가능성 ↑

**장점:**
- **해석 가능**: 의사결정 과정을 시각적으로 볼 수 있음
- **빠른 예측**: 몇 번의 질문만으로 결정
- **비선형 관계**: 복잡한 패턴도 학습 가능

In [ ]:
# Decision Tree 모델 생성 및 학습
dt = DecisionTreeClassifier(
    max_depth=5,        # 트리 깊이 제한 (과적합 방지)
    min_samples_split=20,  # 분할을 위한 최소 샘플 수
    min_samples_leaf=10,   # 리프 노드의 최소 샘플 수
    random_state=42
)

dt.fit(X_train, y_train)

# 예측 수행
y_pred_dt = dt.predict(X_test)

# 정확도 계산
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree 정확도: {accuracy_dt:.3f} ({accuracy_dt*100:.1f}%)")

# 특징 중요도 확인
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': dt.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n특징 중요도:")
for _, row in feature_importance.iterrows():
    print(f"{row['feature']:12}: {row['importance']:.3f} ({'='*int(row['importance']*50)})")

### 4.2 Decision Tree 시각화

실제 모델이 어떤 질문들을 통해 장르를 분류하는지 살펴보자.

In [ ]:
# Decision Tree 시각화
plt.figure(figsize=(20, 12))
plot_tree(dt, 
          feature_names=features,
          class_names=genres,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Decision Tree Visualization', fontsize=16)
plt.show()

# 특징 중요도 시각화
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
bars = plt.bar(feature_importance['feature'], feature_importance['importance'],
               color=['skyblue', 'lightgreen', 'lightcoral', 'gold'], alpha=0.7)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance in Decision Tree')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# 막대 위에 값 표시
for bar, importance in zip(bars, feature_importance['importance']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{importance:.3f}', ha='center', va='bottom')

# 원형 차트로도 표시
plt.subplot(1, 2, 2)
plt.pie(feature_importance['importance'], 
        labels=feature_importance['feature'],
        autopct='%1.1f%%',
        colors=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
plt.title('Feature Importance Distribution')

plt.tight_layout()
plt.show()

print(f"\n해석:")
most_important = feature_importance.iloc[0]
print(f"가장 중요한 특징: {most_important['feature']} ({most_important['importance']:.3f})")
print(f"이 특징이 장르 분류에 가장 큰 영향을 미칩니다.")

## 5. 모델 성능 평가

### 5.1 정확도와 혼동행렬

**정확도(Accuracy)**: 전체 예측 중 맞힌 비율
- 정확도 = (올바른 예측 수) / (전체 예측 수)

**혼동행렬(Confusion Matrix)**: 실제 vs 예측의 상세한 분석
- 어떤 장르를 어떤 장르로 잘못 분류했는지 확인 가능

In [ ]:
# 최적 KNN 모델로 다시 학습
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train, y_train)
y_pred_knn_best = knn_best.predict(X_test)

# 두 모델의 성능 비교
print("모델 성능 비교:")
print(f"KNN (K={best_k}): {accuracy_score(y_test, y_pred_knn_best):.3f} ({accuracy_score(y_test, y_pred_knn_best)*100:.1f}%)")
print(f"Decision Tree: {accuracy_dt:.3f} ({accuracy_dt*100:.1f}%)")

# 혼동행렬 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# KNN 혼동행렬
cm_knn = confusion_matrix(y_test, y_pred_knn_best)
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues',
           xticklabels=genres, yticklabels=genres, ax=axes[0])
axes[0].set_title(f'KNN Confusion Matrix\n(Accuracy: {accuracy_score(y_test, y_pred_knn_best):.3f})')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Decision Tree 혼동행렬
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens',
           xticklabels=genres, yticklabels=genres, ax=axes[1])
axes[1].set_title(f'Decision Tree Confusion Matrix\n(Accuracy: {accuracy_dt:.3f})')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

# 장르별 정확도 분석
print(f"\n장르별 성능 분석 (KNN):")
knn_report = classification_report(y_test, y_pred_knn_best, output_dict=True)
for genre in genres:
    precision = knn_report[genre]['precision']
    recall = knn_report[genre]['recall']
    f1 = knn_report[genre]['f1-score']
    print(f"{genre:10} - Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")

### 5.2 실제 예측 해보기

새로운 음원 데이터를 만들어서 우리 모델이 어떻게 예측하는지 확인해보자.

In [ ]:
# 새로운 음원 데이터 생성 (각 장르의 전형적인 특징)
new_tracks = pd.DataFrame({
    'tempo': [125, 140, 128, 85],
    'energy': [0.7, 0.8, 0.85, 0.3],
    'danceability': [0.75, 0.5, 0.8, 0.2],
    'valence': [0.6, 0.5, 0.65, 0.4]
}, index=['Track_A', 'Track_B', 'Track_C', 'Track_D'])

print("새로운 음원들:")
print(new_tracks)

# KNN과 Decision Tree로 예측
knn_predictions = knn_best.predict(new_tracks)
dt_predictions = dt.predict(new_tracks)

# 예측 확률도 확인 (KNN)
knn_probs = knn_best.predict_proba(new_tracks)
dt_probs = dt.predict_proba(new_tracks)

# 결과 출력
print(f"\n예측 결과:")
results_df = pd.DataFrame({
    'KNN_예측': knn_predictions,
    'DT_예측': dt_predictions,
    'KNN_신뢰도': [max(prob) for prob in knn_probs],
    'DT_신뢰도': [max(prob) for prob in dt_probs]
}, index=new_tracks.index)

print(results_df)

# 각 트랙별 상세 확률 분포
print(f"\n장르별 예측 확률 (KNN):")
prob_df = pd.DataFrame(knn_probs, 
                      columns=knn_best.classes_, 
                      index=new_tracks.index)
print(prob_df.round(3))

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, track in enumerate(new_tracks.index):
    probs = knn_probs[idx]
    bars = axes[idx].bar(genres, probs, color=['red', 'blue', 'green', 'purple'], alpha=0.7)
    axes[idx].set_title(f'{track} - KNN Prediction: {knn_predictions[idx]}')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_ylim(0, 1)
    
    # 막대 위에 확률 표시
    for bar, prob in zip(bars, probs):
        axes[idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                      f'{prob:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n해석:")
for i, track in enumerate(new_tracks.index):
    prediction = knn_predictions[i]
    confidence = max(knn_probs[i])
    print(f"{track}: {prediction} ({confidence:.1%} 확신)")

## 6. 모델 개선하기

### 6.1 특징 정규화 (Feature Scaling)

**왜 정규화가 필요한가?**
- KNN은 거리를 기반으로 하므로 특징들의 스케일이 다르면 문제가 생긴다
- 예: tempo(80-160) vs energy(0-1) → tempo가 거리에 더 큰 영향

**StandardScaler**: 평균 0, 표준편차 1로 정규화

In [ ]:
# 특징 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("정규화 전후 비교:")
print("\n정규화 전:")
print(pd.DataFrame(X_train).describe().round(3))
print("\n정규화 후:")
print(pd.DataFrame(X_train_scaled, columns=features).describe().round(3))

# 정규화된 데이터로 KNN 재학습
knn_scaled = KNeighborsClassifier(n_neighbors=best_k)
knn_scaled.fit(X_train_scaled, y_train)
y_pred_knn_scaled = knn_scaled.predict(X_test_scaled)

accuracy_knn_scaled = accuracy_score(y_test, y_pred_knn_scaled)

print(f"\n성능 비교:")
print(f"KNN (정규화 전): {accuracy_score(y_test, y_pred_knn_best):.3f} ({accuracy_score(y_test, y_pred_knn_best)*100:.1f}%)")
print(f"KNN (정규화 후): {accuracy_knn_scaled:.3f} ({accuracy_knn_scaled*100:.1f}%)")
print(f"Decision Tree:    {accuracy_dt:.3f} ({accuracy_dt*100:.1f}%)")

improvement = accuracy_knn_scaled - accuracy_score(y_test, y_pred_knn_best)
if improvement > 0:
    print(f"\n정규화로 성능 개선: +{improvement:.3f} (+{improvement*100:.1f}%p)")
else:
    print(f"\n정규화 효과: {improvement:.3f} ({improvement*100:.1f}%p)")

### 6.2 교차 검증 (Cross Validation)

**교차 검증이란?**
- 데이터를 여러 번 나누어 더 신뢰할 수 있는 성능 평가
- 5-Fold CV: 데이터를 5등분하여 5번 평가 후 평균

**장점:**
- 더 안정적인 성능 추정
- 과적합 여부 확인 가능

In [ ]:
# 교차 검증으로 성능 평가
print("5-Fold 교차 검증 수행 중...")

# KNN (정규화 적용)
knn_cv_scores = cross_val_score(knn_scaled, X_train_scaled, y_train, cv=5, scoring='accuracy')

# Decision Tree
dt_cv_scores = cross_val_score(dt, X_train, y_train, cv=5, scoring='accuracy')

print(f"\n교차 검증 결과:")
print(f"\nKNN (K={best_k}, 정규화):")
print(f"  각 Fold 점수: {knn_cv_scores.round(3)}")
print(f"  평균: {knn_cv_scores.mean():.3f} ± {knn_cv_scores.std():.3f}")

print(f"\nDecision Tree:")
print(f"  각 Fold 점수: {dt_cv_scores.round(3)}")
print(f"  평균: {dt_cv_scores.mean():.3f} ± {dt_cv_scores.std():.3f}")

# 시각화
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
models = ['KNN\n(Scaled)', 'Decision\nTree']
means = [knn_cv_scores.mean(), dt_cv_scores.mean()]
stds = [knn_cv_scores.std(), dt_cv_scores.std()]

bars = plt.bar(models, means, yerr=stds, capsize=5,
               color=['skyblue', 'lightgreen'], alpha=0.7, edgecolor='black')
plt.ylabel('Accuracy')
plt.title('Cross-Validation Results')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3, axis='y')

# 막대 위에 값 표시
for bar, mean, std in zip(bars, means, stds):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
            f'{mean:.3f}\n±{std:.3f}', ha='center', va='bottom')

# 점수 분포
plt.subplot(1, 2, 2)
plt.boxplot([knn_cv_scores, dt_cv_scores], labels=models)
plt.ylabel('Accuracy')
plt.title('CV Score Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 최종 성능 평가
print(f"\n최종 성능 종합:")
print(f"{'Model':<15} {'Test Accuracy':<15} {'CV Mean':<10} {'CV Std':<10}")
print("-" * 50)
print(f"{'KNN (Scaled)':<15} {accuracy_knn_scaled:<15.3f} {knn_cv_scores.mean():<10.3f} {knn_cv_scores.std():<10.3f}")
print(f"{'Decision Tree':<15} {accuracy_dt:<15.3f} {dt_cv_scores.mean():<10.3f} {dt_cv_scores.std():<10.3f}")

# 최고 모델 선정
if accuracy_knn_scaled > accuracy_dt:
    print(f"\nBest Model: KNN (K={best_k}, with scaling)")
    print(f"   Test Accuracy: {accuracy_knn_scaled:.3f} ({accuracy_knn_scaled*100:.1f}%)")
else:
    print(f"\nBest Model: Decision Tree")
    print(f"   Test Accuracy: {accuracy_dt:.3f} ({accuracy_dt*100:.1f}%)")

## 오늘 배운 머신러닝 개념 정리

### 지도학습 기초
- **분류 문제**: 입력 특징 → 카테고리 예측
- **Train/Test 분할**: 학습용 데이터와 평가용 데이터 분리
- **과적합/과소적합**: 모델 복잡도와 성능의 균형

### K-Nearest Neighbors (KNN)
- **직관적**: "비슷한 것끼리 같은 그룹"
- **K값 조정**: 이웃 개수가 성능에 큰 영향
- **거리 기반**: 특징 정규화가 중요

### Decision Tree
- **해석 가능**: 의사결정 과정을 시각화
- **특징 중요도**: 어떤 특징이 중요한지 확인 가능
- **과적합 주의**: 트리 깊이 제한 필요

### 모델 평가
- **정확도**: 전체 중 맞힌 비율
- **혼동행렬**: 어떤 실수를 했는지 상세 분석
- **교차 검증**: 더 신뢰할 수 있는 성능 평가

### 특징 전처리
- **정규화**: 특징들의 스케일 통일
- **StandardScaler**: 평균 0, 표준편차 1로 변환

### 핵심 포인트
- **데이터 탐색**이 먼저: 시각화로 패턴 파악
- **여러 모델 비교**: 문제에 따라 최적 모델이 다름
- **성능 vs 해석성**: KNN은 성능, Decision Tree는 해석성
- **검증의 중요성**: 교차 검증으로 신뢰성 확보